In [1]:
import os
import sys
sys.path.append(".")
sys.path.append("..")
sys.path.append("...")
import json
import uuid
import cupy as cp
from cuvs.neighbors import cagra, hnsw, brute_force, ivf_flat
from tqdm import tqdm
from datetime import datetime
import time

import faiss  
import numpy as np

In [9]:
embed_cache_path = "data/embedding_cache_level0"
image_names = os.listdir(embed_cache_path)

In [10]:
def load_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
        return data

def load_embeddings(embed_cache_path, wsi_name):
    embeddings_path = os.path.join(embed_cache_path, wsi_name, "embeddings.json")
    embeddings = load_json_file(embeddings_path)
    return embeddings

In [11]:



embeddings = [load_embeddings(embed_cache_path, wsi_name) for wsi_name in tqdm(image_names, desc=f"Batch index:", ascii=True, ncols=150)]
embeddings = [item for sublist in embeddings for item in sublist]  # Flatten the list
embeddings = cp.array(embeddings, dtype=cp.float32)

Batch index:: 100%|###############################################################################################| 124/124 [1:01:12<00:00, 29.61s/it]


In [12]:
# 更新 infos
batch_infos = []
for wsi_name in image_names:
    patch_info_path = os.path.join(embed_cache_path, wsi_name, "patch_info_edited.json")
    if not os.path.exists(patch_info_path):
        patch_info_path_edited = os.path.join(embed_cache_path, wsi_name, 'patch_info_edited.json')
        patch_info_path_origin = os.path.join(embed_cache_path, wsi_name, 'patch_info.json')
        with open(patch_info_path_origin, 'r') as f:
            patch_info_origin = json.load(f)
            # 在最前面加上图片名
            patch_info_edited = [wsi_name + i for i in patch_info_origin]       
        # 将修改后的patch_info写回文件
        with open(patch_info_path_edited, 'w') as f:
            json.dump(patch_info_edited, f)

    patch_infos = load_json_file(patch_info_path)
    batch_infos += patch_infos

In [13]:
cuda_infos_dir = "data/cuda_infos_level0"
if not os.path.exists(cuda_infos_dir):
    os.makedirs(cuda_infos_dir)

cuda_npy_dir = "data/cuda_npy_level0"
if not os.path.exists(cuda_npy_dir):
    os.makedirs(cuda_npy_dir)

def save_infos_to_file(infos, batch_index):
    """将 infos 保存到文件。"""
    file_path = os.path.join(cuda_infos_dir, f"infos_batch_{batch_index}.json")
    with open(file_path, "w") as f:
        json.dump(infos, f)
    
def save_embeddings_to_file(embeddings, batch_index):
    """将 cuda 数组保存到文件。"""
    file_path = os.path.join(cuda_npy_dir, f"embeddings_batch_{batch_index}.npy")
    cp.save(file_path, embeddings)

In [14]:
save_embeddings_to_file(embeddings, "all")
save_infos_to_file(batch_infos, "all")

: 